In [2]:
import os
import pandas as pd

In [1]:
log_path = "/Users/lancer/IdeaProjects/otus_java_2019_10/hw3_gc_comparison/gc_logs"

In [36]:
with open(os.path.join(log_path, 'serial.log')) as f:
    dat = f.readlines()[4:-2]

In [47]:
def get_df(fpath):
    with open(fpath) as f:
        dat = f.readlines()[4:-2]
        
    def parser(line):
        raw = line.split(',')
        gc = raw[0]
        reason = raw[1].replace("Action: ", "")
        causetime = raw[2].replace("gcCause:", "").replace(" ms", "").strip().split("->")
        cause = causetime[0].strip()
        time = int(causetime[1].strip())

        return [gc, reason.strip(), cause, time]
    dataraw = [parser(i) for i in dat]
    return pd.DataFrame.from_records(dataraw, columns=['gc', 'reason', 'cause', 'duration'])

In [48]:
serial = get_df(os.path.join(log_path, 'serial.log'))

In [49]:
g1 = get_df(os.path.join(log_path, 'G1.log'))

In [50]:
parallel = get_df(os.path.join(log_path, 'parallel.log'))

In [64]:
def get_united_stat(df_list, labels):
    number_stat = []
    cat_stat = []
    for df, label in zip(df_list, labels):
        cs = df.describe(include=['O'])
        ns = df.describe()
        cs.columns = [label+"_"+c for c in cs.columns]
        ns.columns = [label+"_"+c for c in ns.columns]
        number_stat.append(ns)
        cat_stat.append(cs)
    return pd.concat(number_stat, axis=1), pd.concat(cat_stat, axis=1)

In [65]:
nsl, csl = get_united_stat([serial, parallel, g1], ["serial", "parallel", "g1"])

In [69]:
csl.to_clipboard()